In [1]:
#from instance_unchecked import Instance, InfeasibleOnBuilding
from instance import Instance, InfeasibleOnBuilding
from merge_apps_util import merge
from model import ModelSolver
from random import random, choice, seed

from time import perf_counter
import pickle


In [2]:
seed(42)

path_apps, tree_apps, graph_apps = [], [], []
for size in range(3,16):
    path_apps.append(f"path_{size:02d}_a")
    tree_apps.append(f"tree_{size:02d}_a")
    tree_apps.append(f"tree_{size:02d}_b")
    tree_apps.append(f"tree_{size:02d}_c")
    tree_apps.append(f"tree_{size:02d}_d")
    graph_apps.append(f"graph_{size:02d}_a")
    graph_apps.append(f"graph_{size:02d}_b")
    graph_apps.append(f"graph_{size:02d}_c")
    graph_apps.append(f"graph_{size:02d}_d")

# for a in path_apps: print(a)
# for a in tree_apps: print(a)
# for a in graph_apps: print(a)

ID = 0
instances = []
for n_size in range(30,61,10):
    for apps_count in range(2,20):

        for _ in range(3):

            n_suffix = choice(['a','b','c','d'])
            n_rp = choice([0,1,2,3,4])
            network_filename = f"DATASET_2/networks/network_{n_size:03d}_{n_suffix}.dat"
            network_rp_filename = f"DATASET_2/networks/network_{n_size:03d}_{n_suffix}_rp_{n_rp}.dat"

            app_filenames, app_rp_filenames = [], []
            for _ in range(apps_count): 
                r = random()
                if r < 0.2: app = choice(path_apps)     # 0.2 prob to choose path
                elif r < 0.6: app = choice(tree_apps)   # 0.4 prob to choose tree
                else: app = choice(graph_apps)          # 0.4 prob to choose graph
                a_rp = choice((0,1,2,3,4))
                app_filenames.append(f"DATASET_2/apps/{app}.dat")
                app_rp_filenames.append(f"DATASET_2/apps/{app}_rp_{a_rp}.dat")

            output_app_filename = f"DATASET_2/apps_merged/id_{ID:04d}.dat"
            output_app_rp_filename = f"DATASET_2/apps_merged/id_{ID:04d}_rp.dat"
            ID += 1

            try:
                merge(app_filenames, app_rp_filenames, output_app_filename, output_app_rp_filename)
            except Exception as e:
                print("ERROR in merge, should not happen!")
            
            instances.append(
                (network_filename, network_rp_filename, output_app_filename, output_app_rp_filename, app_filenames, app_rp_filenames)
            )

for i in instances:
    print(i)

with open('instances_dataset2.pickle', 'wb') as f:
    pickle.dump(instances, f, protocol=pickle.HIGHEST_PROTOCOL)

('DATASET_2/networks/network_030_a.dat', 'DATASET_2/networks/network_030_a_rp_0.dat', 'DATASET_2/apps_merged/id_0000.dat', 'DATASET_2/apps_merged/id_0000_rp.dat', ['DATASET_2/apps/graph_06_d.dat', 'DATASET_2/apps/path_04_a.dat'], ['DATASET_2/apps/graph_06_d_rp_1.dat', 'DATASET_2/apps/path_04_a_rp_4.dat'])
('DATASET_2/networks/network_030_a.dat', 'DATASET_2/networks/network_030_a_rp_4.dat', 'DATASET_2/apps_merged/id_0001.dat', 'DATASET_2/apps_merged/id_0001_rp.dat', ['DATASET_2/apps/tree_03_b.dat', 'DATASET_2/apps/tree_11_a.dat'], ['DATASET_2/apps/tree_03_b_rp_0.dat', 'DATASET_2/apps/tree_11_a_rp_4.dat'])
('DATASET_2/networks/network_030_a.dat', 'DATASET_2/networks/network_030_a_rp_4.dat', 'DATASET_2/apps_merged/id_0002.dat', 'DATASET_2/apps_merged/id_0002_rp.dat', ['DATASET_2/apps/path_13_a.dat', 'DATASET_2/apps/tree_10_a.dat'], ['DATASET_2/apps/path_13_a_rp_4.dat', 'DATASET_2/apps/tree_10_a_rp_4.dat'])
('DATASET_2/networks/network_030_c.dat', 'DATASET_2/networks/network_030_c_rp_0.dat

In [3]:
RESULTS_ROOT_NODE_RELAXATION = [None] * 500
RESULTS_COMPACT = [None] * 500

In [4]:
for idx,i in enumerate(instances):
    
    #if idx not in [18,23,24,25,27,28,29,35,60,61,62,67,70,71,72,73,74,75,76,77,79,80,82,83]:
    #    continue

    print("\n" + ("-"*80) + "\n")
    print(len(i[4]), i)
    try:
        instance = Instance.build(
            network_filename=i[0], 
            network_rp_filename=i[1], 
            app_filename=i[2], 
            app_rp_filename=i[3]
        )
    except InfeasibleOnBuilding:
        print("INFEASIBLE on building")
        RESULTS_ROOT_NODE_RELAXATION[idx] = "INFEASIBLE on building"
        RESULTS_COMPACT[idx] = "INFEASIBLE on building"
        continue

    # print("\nCONTINUOUS RELAXATION :\n")
    # ModelSolver.optimize_model_continuous_relaxation(instance)

    print("\nROOT NODE RELAXATION :\n")
    RESULTS_ROOT_NODE_RELAXATION[idx] = ModelSolver.optimize_model_at_root_node(instance)

    if RESULTS_ROOT_NODE_RELAXATION[idx] is not None and len(RESULTS_ROOT_NODE_RELAXATION[idx]) == 3:
        assert RESULTS_ROOT_NODE_RELAXATION[idx][1] == "OPTIMAL"

        print("\nCOMPACT MODEL :\n")
        RESULTS_COMPACT[idx] = ModelSolver.optimize_model(instance)


--------------------------------------------------------------------------------

2 ('DATASET_2/networks/network_030_a.dat', 'DATASET_2/networks/network_030_a_rp_0.dat', 'DATASET_2/apps_merged/id_0000.dat', 'DATASET_2/apps_merged/id_0000_rp.dat', ['DATASET_2/apps/graph_06_d.dat', 'DATASET_2/apps/path_04_a.dat'], ['DATASET_2/apps/graph_06_d_rp_1.dat', 'DATASET_2/apps/path_04_a_rp_4.dat'])

ROOT NODE RELAXATION :

solve time : 0.004897470795185654, OPTIMAL, z* = 401.3405510302317

COMPACT MODEL :

0.045 s is the total build time of the model
solve time : 0.041996002197265625, OPTIMAL, # explored nodes : 1.0, z* = 424.0

--------------------------------------------------------------------------------

2 ('DATASET_2/networks/network_030_a.dat', 'DATASET_2/networks/network_030_a_rp_4.dat', 'DATASET_2/apps_merged/id_0001.dat', 'DATASET_2/apps_merged/id_0001_rp.dat', ['DATASET_2/apps/tree_03_b.dat', 'DATASET_2/apps/tree_11_a.dat'], ['DATASET_2/apps/tree_03_b_rp_0.dat', 'DATASET_2/apps/tree_1

RuntimeError: optimize_model : unexpected status : 11

In [ ]:
# for idx,r in enumerate(RESULTS):
#     if r is not None:
#         print(f"RESULTS[{idx}] = {r}")